In [1]:
from sympy import symbols, simplify, hessian, solveset, S, solve, log, And, Le, Ge, Eq, Lt, Gt, nonlinsolve, latex, log, Wild, expand_log, logcombine, evaluate,oo, limit,ask, Q, floor, ceiling, lambdify, Sum
from IPython.display import display, HTML, Math

In [2]:
delta_b_negative = True

In [3]:
assets = ['b', 's']  # buying and selling assets
base_symbols = ['s', 'v', 'b', 'w', 'j', 'e', 'Delta', 'a', 'min'] 
# spot price, virtual liquidity, balance, weight, jump size, exponent, delta, anchor price, amm-price, jump-multiplier

all_symbols = {}

for asset in assets:
    temp_dict = {}
    for base in base_symbols:
        var_name = f"{base}_{asset}"
        if base == 'e':
            symbol_obj = symbols(var_name, integer=True)
        elif base == 'b':
            symbol_obj = symbols(var_name, nonnegative=True, integer=True)
        elif delta_b_negative and var_name == 'Delta_b':
            symbol_obj = symbols(var_name, negative=True, integer=True)
        else:
            symbol_obj = symbols(var_name, positive=True, integer=True)
        temp_dict[var_name] = symbol_obj
        # Define the variable in the global namespace
        globals()[var_name] = symbol_obj
    all_symbols[asset] = temp_dict.values()
all_symbols

{'b': dict_values([s_b, v_b, b_b, w_b, j_b, e_b, Delta_b, a_b, min_b]),
 's': dict_values([s_s, v_s, b_s, w_s, j_s, e_s, Delta_s, a_s, min_s])}

In [4]:
simp = True # let's leave this true for now because of the way we calculate the piecewise discrete "derivative" below. Seems to result in vastly simpler equations.

### TODO either show convexity for everything or verify optimality otherwise (i.e. by simply plugging into offchain-tests)

## Target function

we want to minimize the effective price, given a set of exponents for the buying and selling asset each.

In [5]:
if delta_b_negative:
  eff = Delta_s / -Delta_b
else:
  eff = Delta_s / Delta_b
eff

-Delta_s/Delta_b

## Equalities

In [6]:
def deltaBySpot_(asset, s,v, b, w, j, e, Delta, a, min):
  f = (s - (v + b) * w) / w
  if delta_b_negative and asset == 'b':
    f = -floor(f)
  else:
    f = ceiling(f)
  if simp:
    f = simplify(f)
  return f

deltaBySpot = {asset: deltaBySpot_(asset, *all_symbols[asset]) for asset in assets}
deltaBySpot['b']

b_b + v_b - floor(s_b/w_b)

In [7]:
def spotByExp_(s, v, b, w, j, e, Delta, a, min):
  f = floor(a * ((1 + 1/j) ** e))
  if simp:
    f = simplify(f)
  return f
spotByExp = {asset: spotByExp_(*all_symbols[asset]) for asset in assets}
spotByExp['b']

floor(a_b*((j_b + 1)/j_b)**e_b)

## Equality-constraints

In [8]:
deltaSpotBound_b = deltaBySpot['b'] - Delta_b # ==! 0
if simp:
    deltaSpotBound_b = simplify(deltaSpotBound_b)
deltaSpotBound_b

-Delta_b + b_b + v_b - floor(s_b/w_b)

In [9]:
deltaSpotBound_s = deltaBySpot['s'] - Delta_s # ==! 0
if simp:
    deltaSpotBound_s = simplify(deltaSpotBound_s)
deltaSpotBound_s

-Delta_s - b_s - v_s + ceiling(s_s/w_s)

In [10]:
spotExpBound_b = spotByExp['b'] - e_b # ==! 0
if simp:
    spotExpBound_b = simplify(spotExpBound_b)
spotExpBound_b

-e_b + floor(a_b*(j_b + 1)**e_b/j_b**e_b)

In [11]:
spotExpBound_s = spotByExp['s'] - e_s # ==! 0
if simp:
    spotExpBound_s = simplify(spotExpBound_s)
spotExpBound_s

-e_s + floor(a_s*(j_s + 1)**e_s/j_s**e_s)

## Inequality-constraints

- value in A0 of buying must not exceed that of selling
- the exponents must adhere to their upper (buying) resp. lower (selling) bounds given by our equation
- need to buy and sell minimum amounts
- cannot buy more than the available balance
- cannot sell more than maxSelling
- the spot prices must not exceed maxInteger
- bonus: the total number of multiplications for both exponentiations must not exceed expLimit (TODO)

### value cannot decrease

In [12]:
a0Buying = s_s * Delta_b # < 0 if delta_b_negative, > 0 otherwise
if delta_b_negative:
    a0Buying = -a0Buying # > 0
a0Buying

-Delta_b*s_s

In [13]:
a0Selling = s_b * Delta_s # > 0
a0Selling

Delta_s*s_b

In [14]:
a0Bound = a0Buying - a0Selling # <=! 0
a0Bound

-Delta_b*s_s - Delta_s*s_b

In [15]:
if simp:
  a0Bound = simplify(a0Bound)
a0Bound

-Delta_b*s_s - Delta_s*s_b

### exponents cannot result in spot prices that are better than the amm-prices

In [16]:
def expBound_(asset, s, v, b, w, j, e, Delta, a, min):
    e_bound = log(w * (v + b) /a, 1 + 1/j) # constant wrt e
    if simp:
        e_bound = simplify(e_bound)
    if asset == 'b':
        f = e - e_bound # <=! 0 (upper bound)
    else:
        f = e_bound - e # <=! 0 (lower bound)
    if simp:
        f = simplify(f)
    return f

expBound = {asset: expBound_(asset, *all_symbols[asset]) for asset in assets}
expBound['b']

e_b - log((w_b*(b_b + v_b))**(1/log((j_b + 1)/j_b))/a_b**(1/log((j_b + 1)/j_b)))

In [17]:
expBound['s']

-e_s + log((w_s*(b_s + v_s))**(1/log((j_s + 1)/j_s))/a_s**(1/log((j_s + 1)/j_s)))

### spot prices cannot exceed maxInteger

In [18]:
max_s, I_max = symbols('max_s I_max', positive=True, integer=True)

maxSpotBound_b = s_b - I_max # <=! 0
maxSpotBound_b

-I_max + s_b

In [19]:
maxSpotBound_s = s_s - I_max # <=! 0
maxSpotBound_s

-I_max + s_s

### need to swap at least minimum amounts

In [20]:
if delta_b_negative:
  minAmntBound_b = min_b + Delta_b # <=! 0
else:
  minAmntBound_b = min_b - Delta_b # <=! 0
minAmntBound_b

Delta_b + min_b

In [21]:
minAmntBound_s = min_s - Delta_s # <=! 0
minAmntBound_s

-Delta_s + min_s

### need to swap at most maximum amounts

In [22]:
if delta_b_negative:
  maxAmntBound_b = -Delta_b - b_b # <=! 0
else:
  maxAmntBound_b = Delta_b - b_b # <=! 0
maxAmntBound_b

-Delta_b - b_b

In [23]:
maxAmntBound_s = Delta_b - max_s # <=! 0
maxAmntBound_s

Delta_b - max_s

### total number of multiplications of both exponentiations cannot exceed expLimit

In [24]:
# expLimit, aka maximum total number of multiplications for both exponents
m_max = symbols('m_max', positive=True, integer=True)

def numMultsLower_(asset, s,v, b, w, j, e, Delta, a, min):
  return floor(log(e, 2))

# the best number of multiplications we can get by increasing e
def bestMultsAhead_(asset, s,v, b, w, j, e, Delta, a, min):
  return ceiling(log(e, 2))

bestMultsAhead = {asset: bestMultsAhead_(asset, *all_symbols[asset]) for asset in assets}
bestMultsAhead['b']

ceiling(log(e_b)/log(2))

In [25]:
# optimistic estimate. This relies on the vague intuition that exponents will be minimal from the other constraints, so best we can do is increase them if this one is violated
optimisticMultsBound = bestMultsAhead['b'] + bestMultsAhead['s'] - m_max # <=! 0
if simp:
  optimisticMultsBound = simplify(optimisticMultsBound)
optimisticMultsBound

-m_max + ceiling(log(e_b)/log(2)) + ceiling(log(e_s)/log(2))

## Lagrangian

In [26]:
# Decision variables
xs = [e_b, e_s, s_b, s_s, Delta_b, Delta_s]

# Equality-Constraints
hs = [
    deltaSpotBound_b,
    deltaSpotBound_s,
    spotExpBound_b,
    spotExpBound_s,
] 

# Inequality-Constraints
gs = [
    expBound['b'],
    expBound['s'],
    maxSpotBound_b,
    maxSpotBound_s,
    minAmntBound_b,
    minAmntBound_s,
    maxAmntBound_b,
    maxAmntBound_s,
    a0Bound,
    optimisticMultsBound,
] 

# Lagrange multipliers for equality- and inequality-constraints
mus = symbols('mu_1:{}'.format(len(hs)+1), nonnegative=True)
lambdas = symbols('lambda_1:{}'.format(len(gs)+1), nonnegative=True)

# Objective function
f = eff

# Lagrangian
L = f + sum([mu * h for mu, h in zip(mus, hs)]) + sum([lambda_ * g for lambda_, g in zip(lambdas, gs)])
# if simp: # not using this anyways
#     L = simplify(L)
L

lambda_1*(e_b - log((w_b*(b_b + v_b))**(1/log((j_b + 1)/j_b))/a_b**(1/log((j_b + 1)/j_b)))) + lambda_10*(-m_max + ceiling(log(e_b)/log(2)) + ceiling(log(e_s)/log(2))) + lambda_2*(-e_s + log((w_s*(b_s + v_s))**(1/log((j_s + 1)/j_s))/a_s**(1/log((j_s + 1)/j_s)))) + lambda_3*(-I_max + s_b) + lambda_4*(-I_max + s_s) + lambda_5*(Delta_b + min_b) + lambda_6*(-Delta_s + min_s) + lambda_7*(-Delta_b - b_b) + lambda_8*(Delta_b - max_s) + lambda_9*(-Delta_b*s_s - Delta_s*s_b) + mu_1*(-Delta_b + b_b + v_b - floor(s_b/w_b)) + mu_2*(-Delta_s - b_s - v_s + ceiling(s_s/w_s)) + mu_3*(-e_b + floor(a_b*(j_b + 1)**e_b/j_b**e_b)) + mu_4*(-e_s + floor(a_s*(j_s + 1)**e_s/j_s**e_s)) - Delta_s/Delta_b

In [27]:
# Equations from stationarity

def discrete_diff(f, n):
  diff = f.subs(n, n+1) - f
  if simp:
    diff = simplify(diff)
  return diff

equations = []
for x in xs:
  dL_dx = discrete_diff(f, x)\
    + sum([discrete_diff(mu * h, x) for mu, h in zip(mus, hs)])\
    + sum([discrete_diff(lambda_ * g, x) for lambda_, g in zip(lambdas, gs)]) # probably slightly better to do it this way instead of simply using L
  equation = Eq(dL_dx, 0)
  # if simp: # results in error from the bestMultsAhead-constraint
  #   equation = simplify(equation)
  display(equation)
  equations.append(equation)

Eq(lambda_1 + lambda_10*(-ceiling(log(e_b)/log(2)) + ceiling(log(e_b + 1)/log(2))) + mu_3*(-floor(a_b*(j_b + 1)**e_b/j_b**e_b) + floor(a_b*j_b**(-e_b - 1)*(j_b + 1)**(e_b + 1)) - 1), 0)

Eq(lambda_10*(-ceiling(log(e_s)/log(2)) + ceiling(log(e_s + 1)/log(2))) - lambda_2 + mu_4*(-floor(a_s*(j_s + 1)**e_s/j_s**e_s) + floor(a_s*j_s**(-e_s - 1)*(j_s + 1)**(e_s + 1)) - 1), 0)

Eq(-Delta_s*lambda_9 + lambda_3 + mu_1*(floor(s_b/w_b) - floor((s_b + 1)/w_b)), 0)

Eq(-Delta_b*lambda_9 + lambda_4 + mu_2*(-ceiling(s_s/w_s) + ceiling((s_s + 1)/w_s)), 0)

Eq(lambda_5 - lambda_7 + lambda_8 - lambda_9*s_s - mu_1 + Delta_s/(Delta_b*(Delta_b + 1)), 0)

Eq(-lambda_6 - lambda_9*s_b - mu_2 - 1/Delta_b, 0)

In [28]:
# Primal feasibility conditions for equality-constraints
for h in hs:
    equation = Eq(h, 0)
    if simp:
        equation = simplify(equation)
    display(equation)
    equations.append(equation)

Eq(Delta_b - b_b - v_b + floor(s_b/w_b), 0)

Eq(Delta_s + b_s + v_s - ceiling(s_s/w_s), 0)

Eq(e_b - floor(a_b*(j_b + 1)**e_b/j_b**e_b), 0)

Eq(e_s - floor(a_s*(j_s + 1)**e_s/j_s**e_s), 0)

In [29]:
# Complementary slackness conditions
for lambda_, g in zip(lambdas, gs):
    equation = Eq(lambda_ * g, 0)
    if simp:
        equation = simplify(equation)
    display(equation)
    equations.append(equation)

Eq(lambda_1*(e_b - log((w_b*(b_b + v_b))**(1/log((j_b + 1)/j_b))/a_b**(1/log((j_b + 1)/j_b)))), 0)

Eq(lambda_2*(e_s - log((w_s*(b_s + v_s))**(1/log((j_s + 1)/j_s))/a_s**(1/log((j_s + 1)/j_s)))), 0)

Eq(lambda_3*(-I_max + s_b), 0)

Eq(lambda_4*(-I_max + s_s), 0)

Eq(lambda_5*(Delta_b + min_b), 0)

Eq(lambda_6*(-Delta_s + min_s), 0)

Eq(lambda_7*(Delta_b + b_b), 0)

Eq(lambda_8*(Delta_b - max_s), 0)

Eq(lambda_9*(Delta_b*s_s + Delta_s*s_b), 0)

Eq(lambda_10*(-m_max + ceiling(log(e_b)/log(2)) + ceiling(log(e_s)/log(2))), 0)

In [30]:
def verify_inequality_constraints(solution, vars, gs):
    subs_dict = dict(zip(vars, solution))
    for g in gs:
        if Gt(g.subs(subs_dict), 0) == True:
            return False
    return True

In [31]:
# Solve
vars = xs + list(mus) + list(lambdas)
solutions = nonlinsolve(equations, vars)
len(solutions)

In [ ]:
solutions_ = [solution for solution in solutions if verify_inequality_constraints(solution, vars, gs)]
len(solutions_)